In [ ]:
import nibabel as nib
import numpy as np
import os

all_data = []
flat_data = []
ages = []
sexes = []
non_zeros = np.load('non_zeros.npy')
fails = 0
rootDir = 'data'
for dirName, subdirList, fileList in os.walk(rootDir):
    if 'T88' in dirName and 't4_files' not in dirName: 
        ds = dirName.split('/')
        sessionId = ds[2]
        metadataFileName = '/'.join(ds[0:3]) + '/' + sessionId + '.txt'
        dataFileName = '/'.join(ds) + '/' + sessionId + '_mpr_n4_anon_111_t88_masked_gfc.img'
        try:
            img = nib.load(dataFileName)
        except FileNotFoundError:
            dataFileName = '/'.join(ds) + '/' + sessionId + '_mpr_n3_anon_111_t88_masked_gfc.img'
        try:
            img = nib.load(dataFileName)
        except FileNotFoundError:
            fails += 1
            continue
        data = img.get_data()
        # all_data.append(data)
        with open(metadataFileName) as f:
            for line in f:
                if "AGE:" in line:
                    ages.append(list(filter(None, line.split()))[1])
                elif "M/F:" in line:
                    sexes.append(list(filter(None, line.split()))[1])
                    break
        
        # creating the non_zero array of indexes 
        # inds = np.where(data.flatten() > 0)
        # if non_zeros.size == 0: 
        #    non_zeros = inds[0]
        # else: 
        #    non_zeros = np.intersect1d(non_zeros, inds[0])
        
        flat_data.append(data.flatten()[non_zeros])
        
        
# all_data = np.asarray(all_data, float)
flat_data_np = np.array(flat_data)

# WARNING: UNCOMMENTING THE FOLLOWING LINES WILL OVERWRITE YOUR BACKUPS

# np.save('flat_data', flat_data_np)
# np.save('ages', ages)
# np.save('sexes', sexes)

In [1]:
import numpy as np

ages = np.load('ages.npy')
#sexes = np.load('sexes.npy')
flat_data_np = np.load('flat_data.npy')

In [2]:
# finding items, that are equal everywhere
notTheSame = np.where(np.not_equal(flat_data_np[0], flat_data_np[1]))[0]
for element in flat_data_np:
    inds = np.where(np.not_equal(flat_data_np[0], flat_data_np[1]))[0]
    notTheSame = np.intersect1d(notTheSame, inds)
    
flat_data_filtered = flat_data_np[:,notTheSame]
flat_data_np = None
notTheSame = None # freeing up some memory
flat_data_filtered.shape

(434, 1658718)

In [3]:
from sklearn.decomposition import IncrementalPCA
# from sklearn.manifold import TSNE

pca = IncrementalPCA(batch_size=75, n_components=70)
model = pca.fit(flat_data_filtered, ages)

In [4]:
transformedData = model.transform(flat_data_filtered, ages)

In [5]:
transformedData.shape

(434, 70)